In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../')
import session

from aocd import get_data

In [243]:
data = get_data(day=4).splitlines()
draws = list(map(int, data.pop(0).split(',')))
boards = np.array([line.split() for line in data if len(line) > 0]).reshape(-1, 5, 5).astype(int)

results = np.zeros(boards.shape)
mask = np.ones(boards.shape[0]).astype(bool)
wins = []

for n in draws:

    results = results + (temp == n)

    row_sum = (results.sum(axis=2) == 5).any(axis=1)
    col_sum = (results.sum(axis=1) == 5).any(axis=1)

    total = mask & (row_sum | col_sum)

    if total.any():

        mask[total] = False
        score = (temp[total] * (1-results[total])).sum(axis=(1,2)) * n
        wins = wins + list(score)

print(f'1 : {wins[0]}')
print(f'2 : {wins[-1]}')

1 : 64084.0
2 : 12833.0
